In [1]:
# Cell 1: install packages
!apt-get update -qq
!apt-get install -y -qq ffmpeg libsndfile1

# pip installs
!pip install -q faster-whisper assemblyai vosk SpeechRecognition pydub soundfile jiwer


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 114.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.3 MB/s 

In [2]:
# Cell 2
import os, tempfile, base64, time, json, shutil, wave
from IPython.display import Javascript
from google.colab.output import eval_js
from pydub import AudioSegment
import requests
import pandas as pd
from jiwer import wer

/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


In [4]:
# Cell 3 — record_audio_base64() and save helper
def record_audio_base64(duration=5):
    js = """
    async function record(duration){
      const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
      const mediaRecorder = new MediaRecorder(stream);
      let chunks = [];
      mediaRecorder.ondataavailable = e => chunks.push(e.data);
      mediaRecorder.start();
      await new Promise(r => setTimeout(r, duration*1000));
      mediaRecorder.stop();
      await new Promise(r => setTimeout(r, 500));
      const blob = new Blob(chunks, {type:'audio/wav'});
      const reader = new FileReader();
      reader.readAsDataURL(blob);
      return new Promise(resolve => reader.onloadend = () => resolve(reader.result.split(',')[1]));
    }
    """
    print(f"[INFO] Recording for {duration}s — allow microphone access.")
    b64 = eval_js(js + f"record({duration})")
    if not b64:
        raise RuntimeError("No audio captured. Check mic permission or reload the page.")
    return b64

def save_base64_to_file(b64str, out_path):
    data = base64.b64decode(b64str)
    with open(out_path, "wb") as f:
        f.write(data)
    return out_path

# Quick interactive call (run this cell and speak)
b64 = record_audio_base64(5)
tmpdir = tempfile.mkdtemp()
raw_path = os.path.join(tmpdir, "raw_recorded.wav")
save_base64_to_file(b64, raw_path)
print("Saved raw audio to:", raw_path)


[INFO] Recording for 5s — allow microphone access.
Saved raw audio to: /tmp/tmpnoh8rqvk/raw_recorded.wav


In [5]:
# Cell 4 — conversion function + quick verification
from pydub import AudioSegment
import soundfile as sf  # for info checks

def convert_to_wav_16k_mono(src_path, dst_path=None):
    if dst_path is None:
        base, _ = os.path.splitext(src_path)
        dst_path = base + "_16k_mono.wav"
    audio = AudioSegment.from_file(src_path)
    audio = audio.set_frame_rate(16000).set_channels(1).set_sample_width(2)  # 16-bit
    audio.export(dst_path, format="wav")
    return dst_path

wav_16k = convert_to_wav_16k_mono(raw_path)
info = sf.info(wav_16k)
print("Converted file:", wav_16k)
print("samplerate, channels, subtype:", info.samplerate, info.channels, info.subtype)


Converted file: /tmp/tmpnoh8rqvk/raw_recorded_16k_mono.wav
samplerate, channels, subtype: 16000 1 PCM_16


In [6]:
# Cell 5 — load faster-whisper with fallback
from faster_whisper import WhisperModel
import torch

MODEL_NAME = "large-v3"
device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float16" if device == "cuda" else "float32"

def load_whisper_model(primary="large-v3", fallback="medium"):
    try:
        print(f"[INFO] Loading {primary} on {device} with {compute_type} ...")
        m = WhisperModel(primary, device=device, compute_type=compute_type)
        print("[OK] Loaded", primary)
        return m
    except Exception as e:
        print("[WARN] Failed to load primary model:", e)
        print("[INFO] Attempting fallback model:", fallback)
        try:
            m = WhisperModel(fallback, device=device, compute_type=compute_type)
            print("[OK] Loaded fallback:", fallback)
            return m
        except Exception as e2:
            print("[ERROR] Failed to load fallback model:", e2)
            raise

model = load_whisper_model(MODEL_NAME, "medium")


[INFO] Loading large-v3 on cuda with float16 ...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

vocabulary.json: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

[OK] Loaded large-v3


In [7]:
# Cell 6 — Faster-Whisper transcription
def transcribe_faster_whisper(model, wav_path, beam_size=5):
    try:
        segments, info = model.transcribe(wav_path, beam_size=beam_size)
        text = " ".join([seg.text.strip() for seg in segments]).strip()
        return text
    except Exception as e:
        print("[ERROR] faster-whisper failed:", e)
        return ""

fw_text = transcribe_faster_whisper(model, wav_16k)
print("Faster-Whisper output:\n", fw_text)


Faster-Whisper output:
 Hi, this is Kaushik. Hi, my name is Mahika. I'm studying in...


In [8]:
# Cell 7 — AssemblyAI via HTTP (robust)
import os
import time
import requests

# ------------------------------
# Directly set your API key here
# ------------------------------
API_KEY = "d8bd003243034a6f91bbf23ab8047773"
os.environ["ASSEMBLYAI_API_KEY"] = API_KEY
headers = {"authorization": API_KEY}

def assemblyai_transcribe_http(wav_path):
    # Upload
    print("[INFO] Uploading to AssemblyAI...")
    with open(wav_path, "rb") as f:
        resp = requests.post(
            "https://api.assemblyai.com/v2/upload",
            headers=headers,
            data=f
        )
    if resp.status_code not in (200, 201):
        print("[ERROR] upload failed:", resp.status_code, resp.text)
        return ""
    upload_url = resp.json().get("upload_url")

    # Create transcript
    resp = requests.post(
        "https://api.assemblyai.com/v2/transcript",
        json={"audio_url": upload_url},
        headers=headers
    )
    if resp.status_code not in (200, 201):
        print("[ERROR] create transcript failed:", resp.status_code, resp.text)
        return ""
    tid = resp.json().get("id")

    # Poll
    poll_url = f"https://api.assemblyai.com/v2/transcript/{tid}"
    print("[INFO] Polling AssemblyAI...")
    while True:
        r = requests.get(poll_url, headers=headers).json()
        if r.get("status") == "completed":
            return r.get("text", "")
        if r.get("status") == "error":
            print("[ERROR] AssemblyAI error:", r.get("error"))
            return ""
        time.sleep(2)

aa_text = assemblyai_transcribe_http(wav_16k)
print("AssemblyAI output:\n", aa_text)

[INFO] Uploading to AssemblyAI...
[INFO] Polling AssemblyAI...
AssemblyAI output:
 Hi, this is Kaushik. My name is Mehak. I'm starting in.


In [9]:
# Cell 8 — Vosk transcription
VOSK_DIR = "vosk-model-small-en-us-0.15"
if not os.path.exists(VOSK_DIR):
    print("[INFO] Downloading Vosk small model (around 50-100MB)...")
    !wget -q https://alphacephei.com/vosk/models/vosk-model-small-en-us-0.15.zip -O /tmp/vosk_small.zip
    !unzip -q /tmp/vosk_small.zip -d .
    !rm /tmp/vosk_small.zip

from vosk import Model as VoskModel, KaldiRecognizer
import json, wave

def transcribe_vosk(wav_path, model_dir=VOSK_DIR):
    if not os.path.exists(model_dir):
        print("[WARN] Vosk model missing, skipping Vosk."); return ""
    wf = wave.open(wav_path, "rb")
    model = VoskModel(model_dir)
    rec = KaldiRecognizer(model, wf.getframerate())
    text = ""
    while True:
        data = wf.readframes(4000)
        if len(data) == 0: break
        if rec.AcceptWaveform(data):
            res = json.loads(rec.Result())
            text += res.get("text","") + " "
    res = json.loads(rec.FinalResult())
    text += res.get("text","")
    return text.strip()

vosk_text = transcribe_vosk(wav_16k)
print("Vosk output:\n", vosk_text)


[INFO] Downloading Vosk small model (around 50-100MB)...
Vosk output:
 hi this is go check mean this man grandstanding and


In [10]:
# Cell 9 — Comparative analysis table
results = {
    "Faster-Whisper": fw_text,
    "AssemblyAI": aa_text,
    "Vosk": vosk_text
}

gt = input("Paste ground truth transcript for WER (leave blank to skip): ").strip()
rows = []
for name, text in results.items():
    w = None
    if gt and text:
        try:
            w = wer(gt, text)
            w = round(w, 3)
        except Exception as e:
            print(f"[WARN] couldn't compute WER for {name}:", e)
    rows.append({"Model": name, "Recognized Text": text, "WER": w})
df = pd.DataFrame(rows)
display(df)

# Display best model if WER available
valid = [r for r in rows if r["WER"] is not None]
if valid:
    best = min(valid, key=lambda x: x["WER"])
    print(f"Best model by WER: {best['Model']} (WER={best['WER']})")
else:
    print("No ground truth provided (or no results) — WER skipped.")


Paste ground truth transcript for WER (leave blank to skip): hi this is kaushik my name is mehak im studying in


,Model,Recognized Text,WER
0,Faster-Whisper,"Hi, this is Kaushik. Hi, my name is Mahika. I'...",0.545
1,AssemblyAI,"Hi, this is Kaushik. My name is Mehak. I'm sta...",0.636
2,Vosk,hi this is go check mean this man grandstandin...,0.727


Best model by WER: Faster-Whisper (WER=0.545)


In [11]:
from IPython.display import display, Javascript
from google.colab import output
import base64, os, tempfile

# -------------------------
# Step 1: JS recorder function
# -------------------------
def record_audio_base64(duration=5):
    js = """
    async function record(duration){
      const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
      const mediaRecorder = new MediaRecorder(stream);
      let chunks = [];
      mediaRecorder.ondataavailable = e => chunks.push(e.data);
      mediaRecorder.start();
      await new Promise(r => setTimeout(r, duration*1000));
      mediaRecorder.stop();
      await new Promise(r => setTimeout(r, 500));
      const blob = new Blob(chunks, {type:'audio/wav'});
      const reader = new FileReader();
      reader.readAsDataURL(blob);
      return new Promise(resolve => reader.onloadend = () => resolve(reader.result.split(',')[1]));
    }
    """
    print(f"[INFO] Recording for {duration}s — allow microphone access.")
    b64 = eval_js(js + f"record({duration})")
    if not b64:
        raise RuntimeError("No audio captured. Check mic permission or reload the page.")
    return b64

# -------------------------
# Step 2: save base64 to wav
# -------------------------
def save_base64_to_file(b64str, out_path):
    data = base64.b64decode(b64str)
    with open(out_path, "wb") as f:
        f.write(data)
    return out_path

# -------------------------
# Step 3: Record all 5 audios
# -------------------------
audio_labels = ["male_voice", "female_voice", "fast_speech", "noisy_background", "soft_voice"]
DURATION = 5  # seconds per recording
tmpdir = tempfile.mkdtemp()
recorded_paths = []

for label in audio_labels:
    input(f"\nPress Enter to record: {label} ({DURATION} sec)")
    b64 = record_audio_base64(DURATION)
    out_path = os.path.join(tmpdir, f"{label}.wav")
    save_base64_to_file(b64, out_path)
    recorded_paths.append(out_path)
    print(f"[INFO] Saved {label} to: {out_path}")

print("\nAll recordings completed. Files saved in:", tmpdir)
recorded_paths


Press Enter to record: male_voice (5 sec)hi this is kaushik 
[INFO] Recording for 5s — allow microphone access.
[INFO] Saved male_voice to: /tmp/tmpp8cd5lgh/male_voice.wav

Press Enter to record: female_voice (5 sec)hi this is mehakk
[INFO] Recording for 5s — allow microphone access.
[INFO] Saved female_voice to: /tmp/tmpp8cd5lgh/female_voice.wav

Press Enter to record: fast_speech (5 sec)hey im studying msc aiml
[INFO] Recording for 5s — allow microphone access.
[INFO] Saved fast_speech to: /tmp/tmpp8cd5lgh/fast_speech.wav

Press Enter to record: noisy_background (5 sec)hi this is kaushik
[INFO] Recording for 5s — allow microphone access.
[INFO] Saved noisy_background to: /tmp/tmpp8cd5lgh/noisy_background.wav

Press Enter to record: soft_voice (5 sec)Paris is beautiful
[INFO] Recording for 5s — allow microphone access.
[INFO] Saved soft_voice to: /tmp/tmpp8cd5lgh/soft_voice.wav

All recordings completed. Files saved in: /tmp/tmpp8cd5lgh


['/tmp/tmpp8cd5lgh/male_voice.wav',
 '/tmp/tmpp8cd5lgh/female_voice.wav',
 '/tmp/tmpp8cd5lgh/fast_speech.wav',
 '/tmp/tmpp8cd5lgh/noisy_background.wav',
 '/tmp/tmpp8cd5lgh/soft_voice.wav']

In [12]:
from pydub import AudioSegment
import os

def convert_to_16k_mono(src_path):
    base, _ = os.path.splitext(src_path)
    dst_path = base + "_16k_mono.wav"
    audio = AudioSegment.from_file(src_path)
    audio = audio.set_frame_rate(16000).set_channels(1).set_sample_width(2)
    audio.export(dst_path, format="wav")
    return dst_path

# Convert all recorded files
audio_files_16k = [convert_to_16k_mono(f) for f in recorded_paths]
audio_files_16k


['/tmp/tmpp8cd5lgh/male_voice_16k_mono.wav',
 '/tmp/tmpp8cd5lgh/female_voice_16k_mono.wav',
 '/tmp/tmpp8cd5lgh/fast_speech_16k_mono.wav',
 '/tmp/tmpp8cd5lgh/noisy_background_16k_mono.wav',
 '/tmp/tmpp8cd5lgh/soft_voice_16k_mono.wav']

In [13]:
from faster_whisper import WhisperModel
from vosk import Model, KaldiRecognizer
import wave, json
import requests, time

# Load Whisper
whisper_model = WhisperModel("large-v3", device="cuda", compute_type="float32")

# Load Vosk
vosk_model = Model(lang="en-us")

# AssemblyAI headers
ASSEMBLYAI_API_KEY = "d8bd003243034a6f91bbf23ab8047773"
headers = {"authorization": ASSEMBLYAI_API_KEY}


vosk-model-small-en-us-0.15.zip: 100%|██████████| 39.3M/39.3M [00:04<00:00, 9.15MB/s]


In [14]:
def transcribe_whisper(wav_path):
    segments, _ = whisper_model.transcribe(wav_path, beam_size=5)
    text = " ".join([s.text.strip() for s in segments])
    return text

In [15]:
def transcribe_vosk(wav_path):
    wf = wave.open(wav_path, "rb")
    rec = KaldiRecognizer(vosk_model, wf.getframerate())
    vosk_text = ""
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            res = json.loads(rec.Result())
            vosk_text += res.get("text", "") + " "
    res = json.loads(rec.FinalResult())
    vosk_text += res.get("text", "")
    return vosk_text.strip()

In [16]:
def transcribe_assembly(wav_path):
    # Upload
    with open(wav_path, "rb") as f:
        resp = requests.post("https://api.assemblyai.com/v2/upload", headers=headers, data=f)
    upload_url = resp.json()["upload_url"]

    # Request transcription
    resp = requests.post("https://api.assemblyai.com/v2/transcript",
                         json={"audio_url": upload_url},
                         headers=headers)
    tid = resp.json()["id"]

    # Polling
    poll_url = f"https://api.assemblyai.com/v2/transcript/{tid}"
    while True:
        r = requests.get(poll_url, headers=headers).json()
        if r.get("status") == "completed":
            return r.get("text", "")
        if r.get("status") == "error":
            return ""
        time.sleep(2)

In [17]:
ground_truths = {
    "male_voice_16k_mono.wav": "hi this is kaushik.",
    "female_voice_16k_mono.wav": "hi this is mehakk.",
    "fast_speech_16k_mono.wav": "hey im studying msc aiml.",
    "noisy_background_16k_mono.wav": "hi this is kaushik.",
    "soft_voice_16k_mono.wav": "Paris is beautiful."
}


In [18]:
from jiwer import wer
import pandas as pd

results = []

for wav_path in audio_files_16k:
    gt = ground_truths[os.path.basename(wav_path)]

    whisper_text = transcribe_whisper(wav_path)
    vosk_text = transcribe_vosk(wav_path)
    assembly_text = transcribe_assembly(wav_path)

    results.append({
        "Audio": os.path.basename(wav_path),
        "Ground Truth": gt,
        "Whisper": whisper_text,
        "Vosk": vosk_text,
        "AssemblyAI": assembly_text,
        "WER_Whisper": round(wer(gt, whisper_text), 3),
        "WER_Vosk": round(wer(gt, vosk_text), 3),
        "WER_AssemblyAI": round(wer(gt, assembly_text), 3)
    })

df = pd.DataFrame(results)
df


,Audio,Ground Truth,Whisper,Vosk,AssemblyAI,WER_Whisper,WER_Vosk,WER_AssemblyAI
0,male_voice_16k_mono.wav,hi this is kaushik.,"Hi, this is Kaushik.",hi this is kaushik,"Hi, this is Kaushik.",0.50,0.25,0.50
1,female_voice_16k_mono.wav,hi this is mehakk.,"Hi, this is Mehak. I'm studying in",hi this is my heck i'm standing and,"Hi, this is Mehak. I'm studying in.",1.25,1.25,1.25
2,fast_speech_16k_mono.wav,hey im studying msc aiml.,"Hey, I'm studying in MSc AML.",hey i'm standing in as yemen,"Here. I am studying in msc, aml.",1.00,1.00,1.20
3,noisy_background_16k_mono.wav,hi this is kaushik.,"Hi, this is Kaushik.",blogging and hi this is kaushik lab lab,"Blah, blah. Hi, this is Kaushik, blah, blah, b...",0.50,1.25,1.75
4,soft_voice_16k_mono.wav,Paris is beautiful.,Paris is beautiful.,that is as if you have different,Paris is beautiful.,0.00,2.00,0.00
